In [ ]:
# Import our dependencies and libraries
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression

# Read in our housing data
housing_data_df= pd.read_csv("../Resources/City_Zhvi_AllHomes_for_ML_Linear_Regression.csv",parse_dates=True)
housing_data_df

In [ ]:
#Make a top city dataframe based on size rank of each city
topcity = housing_data_df[['SizeRank', 'RegionName']].copy()
topcity

In [ ]:
#Reduce the top city dataframe to the top 60 cities by size rank
topcity = topcity.loc[0:59,'RegionName']

In [ ]:
#Covert city dataframe to list
topcity.values.tolist()

In [ ]:
## start dropping unneeded columns
## dropped RegionID, State, Metro, CountyName 
housing_data_df=housing_data_df.drop(housing_data_df.iloc[:, 0:3],axis = 1)
housing_data_df.head()

In [ ]:
## drop remaining unneeded columns
## dropped all months before January 2010
housing_data_df=housing_data_df.drop(housing_data_df.iloc[:, 1:174],axis = 1)
housing_data_df.head()

In [ ]:
#Dropping all but first value in RegionName. City names appear more than once if there are cities in diffrent states 
# with the same name. Since we are looking for top cities we are ok to drop all occurances after the first.
housing_data_df.drop_duplicates(subset=['RegionName'], inplace=True)
housing_data_df

In [ ]:
#keep only rows with the cities in our top cities by size
housing_topcity_df = housing_data_df[housing_data_df['RegionName'].isin(topcity)]
housing_topcity_df

In [ ]:
## transpose dates to rows
housing_topcity_df=housing_topcity_df.transpose()
housing_topcity_df.head()

In [ ]:
#move first row to column names
housing_topcity_df.columns = housing_topcity_df.iloc[0]
housing_topcity_df

In [ ]:
#remove first row shwowing city name
housing_topcity_df = housing_topcity_df.iloc[1: , :]
housing_topcity_df

In [ ]:
# Reset index
housing_topcity_df = housing_topcity_df.reset_index()
housing_topcity_df.head()

In [ ]:
#Rename date column to date
housing_topcity_df.rename(columns={ housing_topcity_df.columns[0]: "Date" })

In [ ]:
#iterate over columns to create a linear regression model for each city for the date range of 2010 to 2020
#create an empty list to hold our house price predictions once generated

housing_predictions = []

#loop through each column after the date column 

for column in housing_topcity_df.columns[1:]:
    
    
    X = housing_topcity_df.index.values.reshape(-1, 1)   

    #convert X values to float type
    x = X.astype(float)

    Y = housing_topcity_df[column].values.reshape(-1, 1)   

#convert Y value to float type
    y = Y.astype(float)   

    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(x, y)  # perform linear regression
    Y_pred = linear_regressor.predict(X)  # make predictions
    
    #predict the value in 6 months from last data point - this will be equal to the average of the next year
    x_new = np.arange(129).reshape((-1, 1)) 
    
    #set the list to append only the last value predcited by the model which is the value 6 months from last actual data point
    y_new = linear_regressor.predict(x_new)
    y_new[-1][-1]
    housing_predictions.append([column, y_new[-1][-1]])

In [ ]:
#View the list of housing predictions generated for 6 months from last actual data point
housing_predictions

In [ ]:
#convert regrresssion prediction results to a dataframe
future_prediction_df = pd.DataFrame(housing_predictions)
future_prediction_df

In [ ]:
#renaming future price predcition column
future_prediction_df.columns = ['City', 'Future Price']
future_prediction_df

In [ ]:
#changing Future Price column to an integer
future_prediction_df['Future Price'] = future_prediction_df['Future Price'].astype(int)
future_prediction_df

In [ ]:
#convert result to CSV file
next_month_prediction_df.to_csv('../Resources/future_housing_prediction.csv')